## 인프런 2020년 새해 다짐 이벤트 댓글 분석
* https://www.inflearn.com/pages/newyear-event-20200102
* 영상 튜토리얼 : [댓글 수백 수천개 분석하기?! [1/5] 이벤트 데이터 크롤링 feat. 인프런 새해 다짐 이벤트 - YouTube](https://www.youtube.com/watch?v=OUSwQk79H8I&list=PLaTc2c6yEwmohRzCxWQqJ7Z9aqc-sQ5gC)

## 기획자나 마케터가 되어 이벤트 댓글을 분석해 보자!
### 내가 만약 수백 수천개의 댓글을 다 읽어봐야 한다면?
### 댓글 속에 제품에 대한 관심을 빈도수로 추출해야 한다면?
* 쇼핑몰에서 제품 관련 이벤트 진행시 어떤 제품을 고객이 선호하는지 알고 싶다면?
* 고객 DB와 연계할 수 있다면 이벤트 혹은 마케팅 세그먼트로 활용해 볼 수도 있지 않을까?

### 향후 마케팅이나 전략을 어떻게 세워야 할까?

## 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline
# 시각화 결과가 선명하게 표시되도록
%config InlineBackend.figure_format = 'retina'

## 시각화를 위한 한글폰트 설정

In [2]:
# Window 한글폰트 설정
plt.rc("font", family="Malgun Gothic")
# Mac 한글폰트 설정
#plt.rc("font", family="AppleGothic")
plt.rc('axes', unicode_minus=False)

## 크롤링한 파일 로드하기

In [45]:
# df 라는 변수에 이벤트 댓글 파일을 로드합니다.
df = pd.read_csv('data/inflearn-event.csv')
df.shape

(2449, 1)

In [46]:
# head 로 미리보기 합니다.
df.head()

,text
0,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
1,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
2,2020년 목표 : 리액트 공부하기
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
4,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!


In [47]:
# tail 로 미리보기 합니다.
df.tail()

,text
2444,"작년 한해도 일이 바쁘다, 야근해서 힘들다는 핑계로 김영한님의 JPA 강의를 또 스..."
2445,저는 졸업을 1년 남기고 있는 컴퓨터공학과 학생입니다. 졸업 프로젝트로 웹/앱 개발...
2446,"에프터 이펙트를 써본 적은 있는데, 매번 기초만 배우다 말았어요. 이걸 할 줄 안다..."
2447,저번에 인프런에서 페이스북 마케팅 강의를 듣고 많은 도움을 받았습니다. 마케팅 업무...
2448,인프런 0호 팀원이에요!\n그동안 서비스 개발 때문에 js 를 많이 했었는데 앞으론...


## 데이터 전처리

### 네트워크 오류 등으로 발생한 중복 입력 값을 제거
* 빈도 수 중복을 방지하기 위해

In [48]:
df.shape

(2449, 1)

In [49]:
# drop_duplicates 를 통해 중복을 제거합니다. 이때 마지막 글을 남깁니다.
df = df.drop_duplicates(keep='last')
df.shape

(2411, 1)

### 원본은 따로 보존

In [50]:
# 전처리 전에 원본을 보존하기 위해 origin_text 라는 컬럼에 복사해 둡니다.

df['origin_text'] = df['text']
df.head()

,text,origin_text
0,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
1,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는...","2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
4,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!
5,2020 년목표: 비전공자(경영)가 전공자(it) 되기!!!,2020 년목표: 비전공자(경영)가 전공자(it) 되기!!!


### 소문자 변환

In [51]:
# "text" 파이썬은 대소문자를 구분하기 때문에 데이터 필터링을 위해 대문자를 모두 소문자로 변경

df['text'] = df['text'].str.lower()

In [52]:
# 같은 의미의 단어를 하나로 통일 예) python => 파이썬
# replace 는 텍스트가 완전히 일치될 때만 사용할 수 있습니다.
# 일부만 일치한다면 str.replace 를 사용하면 원하는 텍스트로 변경이 가능합니다.

df['text'] = df['text'].str.replace('python', '파이썬')
df[df['text'].str.contains('파이썬')].head()

,text,origin_text
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
18,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요
21,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드
25,"#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 ...","#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 ..."
29,자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 파이썬이 궁금합니다.\n#관심...,자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 Python이 궁금합니다.\n...


In [53]:
lang = {
    'python': '파이썬',
    'java': '자바',
    'react': '리액트',
    'node': '노드',
    'javascript': '자바스크립트',
}
lang

{'python': '파이썬',
 'java': '자바',
 'react': '리액트',
 'node': '노드',
 'javascript': '자바스크립트'}

In [54]:
for lang_name in lang:
    print(lang_name, lang[lang_name])
    df['text'] = df['text'].str.replace(lang_name, lang[lang_name])

python 파이썬
java 자바
react 리액트
node 노드
javascript 자바스크립트


In [55]:
df[df['text'].str.contains('파이썬|자바|리액트')].head(10)

,text,origin_text
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
6,"2020년 목표 : 리액트 공부하기, 스프링 공부하기","2020년 목표 : 리액트 공부하기, 스프링 공부하기"
7,2020년에는 아이티 마스터가 되기 관심강의 리액트\n#관심강의 : 리액트로 노드b...,2020년에는 아이티 마스터가 되기 관심강의 리액트\n#관심강의 : React로 N...
8,2020년에는 아이티 마스터가 되기 관심강의 리액트,2020년에는 아이티 마스터가 되기 관심강의 리액트
12,2020년 목표 : 리액트 공부하기,2020년 목표 : 리액트 공부하기
17,"자바 공부 마스터 하고 싶습니다. 더 자바, 코드를 조작하는 다양한 방법","자바 공부 마스터 하고 싶습니다. 더 자바, 코드를 조작하는 다양한 방법"
18,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요
21,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드
22,querydsl 시리즈를 통해 회사 팀원 교육을 진행할 생각입니다.\n#관심강의 :...,QueryDSL 시리즈를 통해 회사 팀원 교육을 진행할 생각입니다.\n#관심강의 :...
23,next.js로 서버사이드 렌더링에 익숙해지고싶네요. \n#관심강의 : 리액트로 ...,Next.js로 서버사이드 렌더링에 익숙해지고싶네요. \n#관심강의 : React...


### 텍스트로 관심 강의 분리

In [56]:
# 이 이벤트에는 "관심강의"라는 텍스트가 있습니다.
# "관심강의"를 기준으로 텍스트를 분리하고 관심강의 뒤에 있는 텍스트를 가져옵니다.
# 대부분 "관심강의"라는 텍스트를 쓰고 뒤에 강의명을 쓰기 때문입니다.
# 전처리한 내용은 실수를 방지하기 위해 "course" 라는 새로운 컬럼에 담습니다.
# "관심 강의", "관심 강좌" 에 대해서도 똑같이 전처리 합니다.
# ":" 특수문자를 빈문자로 바꿔줍니다.

df['course'] = df['text'].map(lambda x: x.split('관심강의')[-1]) # 관심강의가 들어가면 뒤에꺼가 가져오고 아니면 문장전체가 나옴
df['course'] = df['course'].map(lambda x: x.split('관심강좌')[-1])
df['course'] = df['course'].map(lambda x: x.split('관심 강좌')[-1])
df['course'] = df['course'].map(lambda x: x.split('관심 강의')[-1])
df['course'] = df['course'].str.replace(':', "")
df['course'] = df['course'].str.strip() # 앞뒤로 공백제거

In [57]:
# "text", "course" 전처리 내용 미리보기

df[['text', 'course']].head(10)

,text,course
0,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...,2020년 목표 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 강...
1,"2020년 목표: c++ 열심히 공부해서, 학교에서 꼭 a 맞기..!! \n관심있는...","2020년 목표 c++ 열심히 공부해서, 학교에서 꼭 a 맞기..!! \n관심있는 ..."
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! 우리를 위한 프로그래밍 파이썬 중급
4,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!,2020년 목표 돌머리 비전공자가 멋진 전공자 되기!
5,2020 년목표: 비전공자(경영)가 전공자(it) 되기!!!,2020 년목표 비전공자(경영)가 전공자(it) 되기!!!
6,"2020년 목표 : 리액트 공부하기, 스프링 공부하기","2020년 목표 리액트 공부하기, 스프링 공부하기"
7,2020년에는 아이티 마스터가 되기 관심강의 리액트\n#관심강의 : 리액트로 노드b...,리액트로 노드bird sns 만들기
8,2020년에는 아이티 마스터가 되기 관심강의 리액트,리액트
9,2020년 더욱 성장하는 올해가 되었으면 합니다 !! ㅎㅎㅎㅎ \n관심강의 : op...,"open cv , dl(tensorflow, keras, etc. ), nlp"
10,올해는 작년보다 더 성장하길\n관심강의: 선형대수학개론,선형대수학개론


## 띄어 쓰기를 제거한 텍스트에서 키워드 추출
* TIOBE 프로그래밍 언어 순위 : [index | TIOBE - The Software Quality Company](https://www.tiobe.com/tiobe-index/?fbclid=IwAR34dJfgDHq2DK0C6X3g8IsUno2NhOiikMyxT6fw9SoyujFhy5FPvQogMoA)

In [58]:
# 특정 키워드가 들어가는 댓글을 찾습니다.
search_keyword = ['머신러닝', '딥러닝', '파이썬', '판다스', '공공데이터',
                  'django', '크롤링', '시각화', '데이터분석', 
                  '웹개발', '엑셀', 'c', '자바', '자바스크립트', 
                  '노드', 'vue', '리액트']

# for 문을 통해 해당 키워드가 있는지 여부를 True, False값으로 표시하도록 합니다.
# 키워드에 따라 컬럼을 새로 만듭니다.
for sword in search_keyword:
    df[sword] = df['text'].str.contains(sword)

In [59]:
# 미리보기 합니다.

# df['머신러닝'].sum()
df[search_keyword].sum()

머신러닝      155
딥러닝        74
파이썬       429
판다스         4
공공데이터      12
django     32
크롤링        59
시각화        30
데이터분석      36
웹개발        58
엑셀         19
c         547
자바        421
자바스크립트     86
노드        137
vue       146
리액트       153
dtype: int64

In [60]:
# 파이썬|공공데이터|판다스 라는 텍스트가 들어가는 데이터가 있는지 찾습니다.

df[df['text'].str.contains('파이썬|공공데이터|판다스')]

,text,origin_text,course,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,시각화,데이터분석,웹개발,엑셀,c,자바,자바스크립트,노드,vue,리액트
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! 우리를 위한 프로그래밍 파이썬 중급,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
18,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
21,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드,파이썬 머신러닝 완벽가이드,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
25,"#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 ...","#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 ...","프로그래밍 시작하기 파이썬 입문, 공공데이터로 파이썬 데이터 분석 시작하기\n파이...",False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False
29,자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 파이썬이 궁금합니다.\n#관심...,자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 Python이 궁금합니다.\n...,"프로그래밍 시작하기 파이썬 입문, 공공데이터로 파이썬 데이터 분석 시작하기",False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,"마음으로는 모든 강의가 듣고싶지만, 한가지씩 해나가고싶습니다. 우선은 파이썬부터 시...","마음으로는 모든 강의가 듣고싶지만, 한가지씩 해나가고싶습니다. 우선은 파이썬부터 시...",파이썬 사용자를 위한 웹개발 입문 a to z django + bootstrap,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False
2432,주니어개발자입니다. 토이프로젝트를 꼭 해보고 싶습니다. 올해엔 자바스크립트기초를 탄...,주니어개발자입니다. 토이프로젝트를 꼭 해보고 싶습니다. 올해엔 자바스크립트기초를 탄...,"파이썬입문과 크롤링기초 부트캠프 [쉽게! 하지만, 견고한 자료로!],노드.js로 웹...",False,False,True,False,False,False,True,False,False,False,False,False,True,True,True,False,False
2435,"올해는 파이썬과 r을 자유롭게 사용하고 싶어요..\n데이타 마이닝과, 업무능률향상 ...","올해는 파이썬과 R을 자유롭게 사용하고 싶어요..\n데이타 마이닝과, 업무능률향상 ...","올해는 파이썬과 r을 자유롭게 사용하고 싶어요..\n데이타 마이닝과, 업무능률향상 ...",False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2447,저번에 인프런에서 페이스북 마케팅 강의를 듣고 많은 도움을 받았습니다. 마케팅 업무...,저번에 인프런에서 페이스북 마케팅 강의를 듣고 많은 도움을 받았습니다. 마케팅 업무...,파이썬 입문 및 웹 크롤링을 활용한 다양한 자동화 어플리케이션 제작하기,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,False


In [62]:
# 결과를 모두 더하면 해당 키워드의 등장 빈도수를 카운트 할 수 있습니다.
# search_keyword 컬럼만 가져와서 빈도수를 sum으로 더합니다.

df[search_keyword].sum().sort_values(ascending=False)

c         547
파이썬       429
자바        421
머신러닝      155
리액트       153
vue       146
노드        137
자바스크립트     86
딥러닝        74
크롤링        59
웹개발        58
데이터분석      36
django     32
시각화        30
엑셀         19
공공데이터      12
판다스         4
dtype: int64

In [82]:
# 공공데이터 텍스트가 들어가는 문장만 찾습니다.
# pandas 를 통해 볼때 문장이 길면 끝까지 보이지 않습니다.
# 문장의 전체를 보기 위해 for문을 통해 해당 텍스트를 순회하며 출력합니다.
# 이 때, 데이터 사이에 ------ 줄로 구분해서 표시하도록 합니다.

pub_data = df.loc[df['text'].str.contains('자바') & ~df['text'].str.contains('자바스크리트'), 'text']
for num, pub_text in enumerate(pub_data[:5]):
    # print("-"=20)  이거 왜 안되냐
    print(num, pub_text)

0 자바 공부 마스터 하고 싶습니다.   더 자바, 코드를 조작하는 다양한 방법
1 querydsl 시리즈를 통해 회사 팀원 교육을 진행할 생각입니다.
#관심강의 : 자바 orm 표준 jpa 프로그래밍
2 자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 파이썬이 궁금합니다.
#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 시작하기
3 자바스크립트를 더 공부하고 싶습니다.
#관심강의 : 함수형 프로그래밍과 자바script es6+ ,함수형 프로그래밍과 자바script es6+ 응용편
4 자바를 공부하려고 가입했습니다. 영역을 더 넓혀 나갑시다~


## 판다스 단어가 들어가는 텍스트만 찾기
* 이미 str.contains 를 통해 판다스가 들어가는 텍스트에 대해 컬럼을 만들어 놨습니다. 이 값이  True 라면 판다스 강좌 입니다.

In [83]:
# pandas 라는 텍스트가 들어가는 내용만 찾습니다.

df[df['course'].str.contains('pandas')]

,text,origin_text,course,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,시각화,데이터분석,웹개발,엑셀,c,자바,자바스크립트,노드,vue,리액트
690,"취미로 안드로이드 개발하는 사람입니다. 자바로 작성하다 보니, kotlin이 안드로...","취미로 안드로이드 개발하는 사람입니다. java로 작성하다 보니, kotlin이 안...","kotlin, pandas, 파이썬, c언어",False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,False
758,pandas 라입러리에 관심이 많아서 배워보려 합니다 #관심강의 : 박조은 - pa...,pandas 라입러리에 관심이 많아서 배워보려 합니다 #관심강의 : 박조은 - pa...,박조은 - pandas,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## 빈도수 계산을 위한 텍스트 데이터 벡터화
* BOW 단어 가방에 단어를 토큰화 해서 담아줌

### BOW(bag of words)
* 가장 간단하지만 효과적이라 널리쓰이는 방법
* 장, 문단, 문장, 서식과 같은 입력 텍스트의 구조를 제외하고 각 단어가 이 말뭉치에 얼마나 많이 나타나는지만 헤아립니다.
* 구조와 상관없이 단어의 출현횟수만 세기 때문에 텍스트를 담는 가방(bag)으로 생각할 수 있습니다.
* BOW는 단어의 순서가 완전히 무시 된다는 단점이 있다. 예를 들어 의미가 완전히 반대인 두 문장이 있다고 합니다.
    - `it's bad, not good at all.` 
    - `it's good, not bad at all.` 
* 위 두 문장은 의미가 전혀 반대지만 완전히 동일하게 반환됩니다.
* 이를 보완하기 위해 n-gram을 사용하는 데 BOW는 하나의 토큰을 사용하지만 n-gram은 n개의 토큰을 사용할 수 있도록 합니다.
* min_df는 문서에 특정 단어가 최소 몇 번 이상 문서에 등장하는 단어를 가방에 담겠다는 의미입니다.

* [Bag-of-words model - Wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)

### 사이킷런의 CountVectorizer를 통해 벡터화
* 미리 전처리 해둔 텍스트 데이터로 벡터화 합니다.
* 모두 소문자로 변환시키기 때문에 영어의 경우 good, Good, gOod이 모두 같은 특성이 됩니다.
* 의미없는 특성을 많이 생성하기 때문에 적어도 두 번이상 문서에 나타난 토큰만을 사용한다.
* [6.2. Feature extraction — scikit-learn 0.22.2 documentation](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
* [sklearn.feature_extraction.text.CountVectorizer — scikit-learn 0.22.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [84]:
# "파이썬 데이터 분석" 이라는 텍스트를 토큰화 split 을 통해 해봅니다.
"파이썬 데이터 분석".split()

['파이썬', '데이터', '분석']

In [90]:
# 사이킷런의 CountVectorizer 를 통해 벡터화 합니다.
# vectorizer 라는 변수에 CountVectorizer 를 담습니다.
# analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
# tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
# preprocessor = None, # 전처리 도구
# stop_words = None, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
# min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋습니다. 
# ngram_range=(3, 6), # BOW의 단위 갯수의 범위를 지정합니다.
# max_features = 2000 # 만들 피처의 수, 단어의 수

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=2000, ngram_range=(3,6))
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=2000, min_df=1,
                ngram_range=(3, 6), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [92]:
# df['course'] 만 vectorizer로 벡터화 합니다. 결과를 feature_vector 변수에 할당합니다.
corpus = df['course']

X = vectorizer.fit_transform(corpus)
# print(vectorizer.get_feature_names())

print(X.toarray())


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [94]:
# vectorizer 에서 get_feature_names 를 추출합니다.
# vocab 이라는 변수에 할당해서 재사용 합니다.
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

2000


['12개 만들면서 배우는',
 '12개 만들면서 배우는 ios',
 '12개 만들면서 배우는 ios 아이폰',
 '12개 만들면서 배우는 ios 아이폰 개발',
 '12개를 만들며 배우는',
 '12개를 만들며 배우는 swift4',
 '12개를 만들며 배우는 swift4 ios11',
 '12개를 만들며 배우는 swift4 ios11 아이폰',
 '2018 do it',
 '2018 do it 안드로이드']

In [96]:
# 각 리뷰마다 등장하는 단어에 빈도수가 표현됩니다. 0 은 등장하지 않음을 의미합니다.
# feature_vector[:10].toarray() 를 가져와 데이터프레임으로 만들고 컬럼명은 vocab 으로 지정합니다.

df_vocab = pd.DataFrame(X.toarray())
df_vocab.columns = vocab
df_vocab.head()

,12개 만들면서 배우는,12개 만들면서 배우는 ios,12개 만들면서 배우는 ios 아이폰,12개 만들면서 배우는 ios 아이폰 개발,12개를 만들며 배우는,12개를 만들며 배우는 swift4,12개를 만들며 배우는 swift4 ios11,12개를 만들며 배우는 swift4 ios11 아이폰,2018 do it,2018 do it 안드로이드,...,활용한 다양한 자동화 어플리케이션,활용한 다양한 자동화 어플리케이션 제작하기,활용한 데이터분석과 it보안,활용한 리액트 native,활용한 리액트 native 개발,활용한 메신져 만들기,활용한 메신져 만들기 android,활용한 파이썬 프로그래밍,활용한 파이썬 프로그래밍 완벽,활용한 파이썬 프로그래밍 완벽 이해
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
# 위에서 구한 단어벡터를  np.sum으로 더하면 단어가 전체에서 등장하는 횟수를 알 수 있습니다.
# 벡터화 된 피처를 확인해 봅니다.
# Bag of words 에 몇 개의 단어가 들어있는지 확인합니다.

df_vocab_sum = pd.DataFrame(df_vocab.sum()).sort_values(by=0, ascending=False)
df_vocab_sum.head(20)

,0
홍정모의 따라하며 배우는,100
노드bird sns 만들기,60
스프링 데이터 jpa,60
리액트로 노드bird sns,49
머신러닝 완벽 가이드,49
리액트로 노드bird sns 만들기,49
파이썬 머신러닝 완벽,49
함수형 프로그래밍과 자바script es6,48
프로그래밍과 자바script es6,48
함수형 프로그래밍과 자바script,48


In [104]:
# 행과 열의 축을 T로 바꿔주고 빈도수로 정렬합니다.

df_vocab_sum.T

,홍정모의 따라하며 배우는,노드bird sns 만들기,스프링 데이터 jpa,리액트로 노드bird sns,머신러닝 완벽 가이드,리액트로 노드bird sns 만들기,파이썬 머신러닝 완벽,함수형 프로그래밍과 자바script es6,프로그래밍과 자바script es6,함수형 프로그래밍과 자바script,...,방법 스프링 데이터 jpa 스프링,방법 스프링 데이터 jpa 스프링 프레임워크,배우고 싶습니다 관심있는,배우는 ios 아이폰,배우는 ios 아이폰 개발,배우는 swift4 ios11,배우는 swift4 ios11 아이폰,배우는 swift4 ios11 아이폰 ios,배우는 swift4 ios11 아이폰 ios 개발,활용한 파이썬 프로그래밍 완벽 이해
0,100,60,60,49,49,49,49,48,48,48,...,2,2,2,2,2,2,2,2,2,2


In [106]:
# ["course", "freq"] 라는 컬럼명을 주어 위에서 만든 데이터프레임을 변환합니다.

df_vocab_count = df_vocab_sum.reset_index()
df_vocab_count.columns = ["course", "freq"] 
df_vocab_count.head()

,course,freq
0,홍정모의 따라하며 배우는,100
1,노드bird sns 만들기,60
2,스프링 데이터 jpa,60
3,리액트로 노드bird sns,49
4,머신러닝 완벽 가이드,49


In [111]:
# 강의명을 토큰 3개로 중복제거하기 위해, 강좌명에서 지식공유자의 이름을 빈문자열로 변경합니다.
# 강의명을 lambda 식을 사용해서 x.split() 으로 나누고 [:4] 앞에서 4개까지만 텍스트를 가져오고 다시 join으로 합쳐줍니다. 
# 중복된 텍스트를 구분해서 보기 위함입니다.

df_vocab_count['course4'] = df_vocab_count['course'].map(lambda x: " ".join(x.split()[:4]))
df_vocab_count['course4'].head()

0      홍정모의 따라하며 배우는
1     노드bird sns 만들기
2        스프링 데이터 jpa
3    리액트로 노드bird sns
4        머신러닝 완벽 가이드
Name: course4, dtype: object

In [115]:
# 3개의 ngram과 빈도수로 역순 정렬을 하게 되면 빈도수가 높고, ngram수가 많은 순으로 정렬이 됨 
# 여기에서 drop_duplicates로 첫 번째 강좌를 남기고 나머지 중복을 삭제 합니다.

df_vocab_count = df_vocab_count.drop_duplicates(['course4'])
df_vocab_count.sort_values(by='freq', ascending=False).head()

,course,freq,course4
0,홍정모의 따라하며 배우는,100,홍정모의 따라하며 배우는
1,노드bird sns 만들기,60,노드bird sns 만들기
2,스프링 데이터 jpa,60,스프링 데이터 jpa
3,리액트로 노드bird sns,49,리액트로 노드bird sns
4,머신러닝 완벽 가이드,49,머신러닝 완벽 가이드


In [ ]:
# 빈도수로 정렬을 하고 어떤 강좌가 댓글에서 가장 많이 언급되었는지 봅니다.



In [ ]:
# 전처리가 다 되었다면 다른 팀 또는 담당자에게 전달하기 위해  csv 형태로 저장합니다.



## TF-IDF 로 가중치를 주어 벡터화
### TfidfTransformer()
* norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정합니다.
    - L2 : 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이고 기본 값(유클리디안거리)
    - L1 : 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절(맨하탄거리)
* smooth_idf=False
    - 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정
* sublinear_tf=False
* use_idf=True
    - TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부

In [119]:
# TfidfTransformer 를 불러와서 가중치를 주어 벡터화 합니다.
# transformer 라는 변수로 저장하고 재사용합니다.


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('count', CountVectorizer(vocabulary=vocab)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
pipe['count'].transform(corpus).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
feauture_tfidf = pipe['tfid'].idf_
feauture_tfidf

In [120]:
# fit_transform 으로 가중치를 적용하고 결과를 feature_tfidf 로 받습니다.
transformer = TfidfTransformer()
transformer.fit_transform

KeyError: 'tfidf'

In [ ]:
pipe.transform(corpus)
feature_tfidf.shape

In [ ]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 one-hot-vector에 TF-IDF 가중치 반영한 결과를 봅니다.
# feature_tfidf.toarray() 로 배열로 만들고  데이터 프레임을 만들어 tfidf_freq 라는 변수에 할당해서 봅니다.



In [ ]:
# tfidf_freq를 sum 으로 가중치를 다 더해줍니다.



In [ ]:
# 중간에 생략되는 단어를 자세히 보고자 할 때
# for문으로 출력해 봅니다.



## 군집화 
### KMeans
* [2.3. Clustering — scikit-learn 0.22.2 documentation](https://scikit-learn.org/stable/modules/clustering.html#k-means)

In [122]:
from sklearn.cluster import KMeans
from tqdm import trange
inertia = []

start = 30
end = 70

# 적절한 클러스터의 갯수를 알기 위해 inertia 값을 구함
# trange 를 통해 시작과 끝 값을 지정해 주면 진행 정도를 알 수 있습니다.
# 학습을 할 때는 feature_tfidf 값을 사용합니다.
# cls.inertia_ 값을 inertia 리스트에 저장합니다.

kmeans = KMeans(radom_state=42, n_jobs=-1, verbose=1)
kmeans.fit(feature_tfidf)

In [ ]:
# 위에서 구한 값을 시각화 합니다.
# x축에는 클러스터의 수를 y축에는 inertia 값을 넣어 그립니다.




* 적정한 클러스터 갯수를 넣어 군집화 합니다.

In [ ]:
# n_clusters 에 적절한 값을 넣어줍니다.
# fit.predict 를 하고 결과를 cluster 라는 새로운 컬럼에 담습니다.



In [ ]:
# df["cluster"] 의 빈도수를 value_counts로 세어봅니다.



### MiniBatchKMeans
* [Comparison of the K-Means and MiniBatchKMeans clustering algorithms — scikit-learn 0.22.1 documentation](https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html)

In [ ]:
# batch_size 를 쓸 수 있는 MiniBatchKMeans 로 군집화
from sklearn.cluster import MiniBatchKMeans
b_inertia = []

# 적절한 클러스터의 갯수를 알기 위해 inertia 값을 구함
# trange 를 통해 시작과 끝 값을 지정해 주면 진행 정도를 알 수 있습니다.
# b_inertia 리스트에 cls.inertia_ 값을 넣어줍니다.


In [ ]:
# 위에서 구한 값을 시각화 합니다.
# x축에는 클러스터의 수를 y축에는 b_inertia 값을 넣어 그립니다.



In [ ]:
# MiniBatchKMeans 를 통해 학습을 시킵니다.
# 결과를 bcluster 라는 변수에 저장합니다.



In [ ]:
# bcluster의 빈도수를 구합니다.



In [ ]:
# 어떤 강좌명이 있는지 특정 클러스터의 값을 봅니다.  


In [ ]:
# bcluster, cluster, course 값을 미리보기 합니다.



### 클러스터 예측 정확도 확인하기

In [ ]:
# n_clusters 위에서 정의한 클러스터 수를 사용
feature_array = feature_vector.toarray()
# 예측한 클러스터의 유니크 값
labels = np.unique(prediction)
df_cluster_score = []
df_cluster = []
for label in labels:
    id_temp = np.where(prediction==label) # 예측한 값이 클러스터 번호와 매치 되는 것을 가져옴
    x_means = np.mean(feature_array[id_temp], axis = 0) # 클러스터의 평균 값을 구함
    sorted_means = np.argsort(x_means)[::-1][:n_clusters] # 값을 역순으로 정렬해서 클러스터 수 만큼 가져옴
    features = vectorizer.get_feature_names()
    best_features = [(features[i], x_means[i]) for i in sorted_means] 
    # 클러스터별 전체 스코어
    df_score = pd.DataFrame(best_features, columns = ['features', 'score'])
    df_cluster_score.append(df_score)
    # 클러스터 대표 키워드
    df_cluster.append(best_features[0])

In [ ]:
# 개별 클러스터에서 점수가 가장 높은 단어를 추출 아래 점수가 클수록 예측 정확도가 높음
# MiniBatchKMeans 로 예측한 값 기준



In [ ]:
# score 정확도가 1이 나온 클러스터를 찾아봄 - 같은 강좌끼리 묶였는지 확인 함



## WordCloud
* [amueller/word_cloud: A little word cloud generator in Python](https://github.com/amueller/word_cloud)
* 설치 방법 : [Wordcloud :: Anaconda Cloud](https://anaconda.org/conda-forge/wordcloud)

In [ ]:
# 불용어 - 자주 등장하지만 의미가 크지 않아 제외하고 싶은 텍스트
# stopwords = ["관심 강의", "관심강의", "관심", "강의", "강좌", "강의를",
#              "올해", "올해는", "열심히", "공부를", "합니다", "하고", "싶어요", 
#              "있는", "있습니다", "싶습니다", "2020년"]
# 불용어를 제거하지 않고 그리려면 아래 주석을 풀어서 stopword 변수에 덮어쓰기를 합니다.
stopwords = []
# displayWordCloud 라는 함수를 만들어 재사용합니다.

